In [ ]:
# Clear all handlers first to start fresh
from loguru import logger
logger.remove()
import torch

from rag_app.services.rag import RagService

from llama_index.readers.file import PyMuPDFReader
from rag_app.services.ingest import IngestionService
from llama_index.core.node_parser import SemanticSplitterNodeParser

from rag_app.embeddings import get_chunk_embeddings, get_embed_model
from openai import AsyncOpenAI
from rag_app.config.settings import settings
from rag_app.core.vector_client import VectorClient
from rag_app.llm import get_llm_model
from sentence_transformers import CrossEncoder


In [ ]:
import nest_asyncio
nest_asyncio.apply()

reader = PyMuPDFReader()
chunk_embed_model = get_chunk_embeddings()
embedding_client = get_embed_model()

llm_model = get_llm_model()

vector_client = await VectorClient.create(
    api_key=settings.pinecone.api_key,
    environment=settings.pinecone.environment,
    index_name=settings.pinecone.index_name,
    dimension=settings.pinecone.dimension,
    metric=settings.pinecone.metric,
    cloud=settings.pinecone.cloud,
    region=settings.pinecone.region,
)

encoder_model = CrossEncoder(settings.reranker.model, device="cpu" if not torch.cuda.is_available() else "cuda")


node_parser = SemanticSplitterNodeParser(embed_model=chunk_embed_model)

ingest_service = IngestionService(
    reader=reader,
    node_parser=node_parser,
    llm_model=llm_model,
    vector_client=vector_client,
    embedding_client=embedding_client,
)

rag_service = RagService(
    embed_model=embedding_client,
    vector_client=vector_client,
    llm_model=llm_model,
    encoder_model=encoder_model)

In [ ]:
# nodes = await ingest_service.ingest(file_path="C:/Users/mrudh/Documents/Data/CWMG-KS-Vol-001-I.pdf")

In [ ]:
query = "What was Gandhi’s early argument in favor of vegetarianism?"
matches = await rag_service.answer_query(query=query)

In [ ]:
print(matches['answer'])

In [ ]:
matches['sources']

In [ ]:
from google import genai

client = genai.Client()

result = client.models.embed_content(
        model="gemini-embedding-001",
        contents="What is the meaning of life?")

print(result.embeddings)

In [1]:
from rag_app.llm import get_llm_model

In [2]:
model = get_llm_model()

2025-10-23 11:18:27.468 | INFO     | rag_app.llm:get_llm_model:18 - Initializing Local LLM
2025-10-23 11:18:27 | INFO     | rag_app.llm:get_llm_model:18 - Initializing Local LLM


In [3]:
import nest_asyncio
nest_asyncio.apply()

# from openai import AsyncOpenAI

# client = AsyncOpenAI(base_url="http://localhost:8080/v1", api_key="test")

from rag_app.schemas.metadata import QuestionMetadata

node = """to the captain. He reproved these pugilistic gentlemen, and ever
since then we had no more rows.
Thus, dividing our time between eating and amusements,
we moved onward.
After two days' voyage, the steamer passed by, but did not
touch,
Gibraltar.
This caused much disappointment,
mostly
among smokers, who wanted to get tobacco, duty-free in Gibral-
tar, as some of us had entertained a hope the steamer would cast
anchor.
The next place reached was Malta. It being a coaling sta-
tion, the steamer stops there for about nine hours. Almost all the
passengers went ashore.
Malta is a beautiful island without the London smoke. The
construction of houses is different. We had a look round the Gover-
nor's palace. The armoury is well worth a visit. Napoleon's car-
riage is on view there. You see there some beautiful paintings
too. The market is not bad. The fruit is cheap. The cathedral is
magnificent.
We had a nice drive of about six miles to the orange gar-
den. There you see some thousands of orange trees and some
ponds with gold fish. The drive was very cheap, only 2s. 6d.
What a wretched place Malta is for beggars! You cannot go
along the road quietly without being pestered by a crowd of dirty-
looking beggars. Some would offer to be your guides, others would
offer to take you to shops where you could buy cigars or the famous
Maltese sweet nougat.
From Malta we reached Brindisi. It is a good harbour and
that is all. You cannot pass a single day in amusement. We had
about nine hours or more at our disposal, but we could not uti-
lize even four.
After Brindisi we reached Port Said.
There we took final
leave of Europe and the Mediterranean. Of course, there is no-
thing to be seen in Port Said, unless you want to see the dregs of
society. It is full of rogues and rascals.
From Port Said the steamer moves along very slowly, for we
enter the Suez Canal of M. de Lesseps. It is a distance of eighty-
seven miles. The steamer took nearly twenty-four hours to travel
that distance. We were close to the land on both sides. The strip
of water is so narrow that two steamers cannot go abreast except
at certain places. At night the sight is charming. All the ships are
required to light electric lights in front and these are very power-
ful. The scene when two pass one another is very pleasant. The
electric light you get from the opposite ship is simply dazzling."""

prompt = f"""
You will be provided with a chunk of text from a document.
Your task is to extract 5 questions that can be answered using the information in the text.
you will answer in the following json format
You should only respond with the JSON object and nothing else.no other text.this is very important.
Only output 5 questions not more not less. 
"""

# input_messages = [
#     {"role": "system", "content": "You are a helpful assistant that extracts questions from text."},
#     {"role": "user", "content": prompt}
#     ]


# result = await model.generate_completion(
#     messages=input_messages,
#     structured_format=QuestionMetadata,
# )

In [1]:
from rag_app.config.settings import settings

In [3]:
settings.llm.provider

'gemini'

In [2]:
import os

os.environ['LLM_PROVIDER'] = 'openai'

In [3]:
settings.llm.provider

'gemini'